In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error as MSE, mean_absolute_error as MAE
from joblib import dump, load

import random
from transformers import *


In [15]:

# Фиксируем датчики случайных чисел
random.seed(42)
np.random.seed(42)

# Загружаем данные
df = pd.read_csv('data.csv')

# Разделяем ИСХОДНЫЕ ДАННЫЕ на тренировочную и тестовую выборки
df_train, df_test = train_test_split(df, train_size=0.75, random_state=42)

/var/folders/34/9lmn_1px5cs289tjl5rn7cs80000gn/T/ipykernel_7799/2594801504.py:6: DtypeWarning: Columns (33,37,38,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [16]:
df_kladr = pd.read_csv('kladr-2.csv')
df_kladr = df_kladr.sort_values("TYPE_NP")
df_kladr = df_kladr.drop_duplicates(keep='first', subset='NAME').sort_values('NAME')

In [17]:
alphabet = {}
for i, j in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "K"]):
    alphabet[i] = j

In [18]:
df_kladr["OKRUG_CODE"] = df_kladr["OKRUG_CODE"].replace(alphabet)

In [19]:
df_kladr.loc[39998, "SOCR"] = 'г'

In [20]:
df_train = df_train.merge(df_kladr[['NAME', 'REGION', 'TYPE_NP', 'OKRUG_OCATD', 'OKRUG_CODE']], left_on='ann_city', right_on='NAME', how='left').drop(columns=['NAME'])
df_train.rename(columns={'TYPE_NP': 'type_city',
                   'REGION': 'region',
                   'OKRUG_OCATD': 'okr_ocatd',
                   'OKRUG_CODE': 'okr_code'
                   }, inplace=True)

In [21]:
df_test = df_test.merge(df_kladr[['NAME', 'REGION', 'TYPE_NP', 'OKRUG_OCATD', 'OKRUG_CODE']], left_on='ann_city', right_on='NAME', how='left').drop(columns=['NAME'])
df_test.rename(columns={'TYPE_NP': 'type_city',
                   'REGION': 'region',
                   'OKRUG_OCATD': 'okr_ocatd',
                   'OKRUG_CODE': 'okr_code'
                   }, inplace=True)

In [22]:
df_train["type_city"] = df_train["type_city"].fillna("Nan")
df_test["type_city"] = df_test["type_city"].fillna("Nan")

In [23]:
df_town = pd.read_excel('town.xlsx')

In [24]:
import pandas as pd

# Добавляем данные из df_3 по численности
df_train = df_train.merge(df_town[['Town', 'Count']], left_on='ann_city', right_on='Town', how='left').drop(columns=['Town'])

# Заменяем значения в type_city по условиям Count
df_train.loc[(df_train['Count'] >= 50) & (df_train['Count'] < 100), 'type_city'] = 'gor_100'
df_train.loc[(df_train['Count'] >= 100) & (df_train['Count'] < 250), 'type_city'] = 'gor_250'
df_train.loc[(df_train['Count'] >= 250) & (df_train['Count'] < 500), 'type_city'] = 'gor_500'
df_train.loc[(df_train['Count'] >= 500) & (df_train['Count'] < 1000), 'type_city'] = 'gor_1000'
df_train.loc[(df_train['Count'] >= 1000) & (df_train['Count'] < 2500), 'type_city'] = 'gor_2500'
df_train.loc[(df_train['Count'] >= 2500) & (df_train['Count'] < 5000), 'type_city'] = 'gor_5000'
df_train.loc[df_train['Count'] > 5000, 'type_city'] = 'gor_10000'
# df_train = df_train.drop(columns=['Count'])

In [25]:
import pandas as pd

# Добавляем данные из df_3 по численности
df_test = df_test.merge(df_town[['Town', 'Count']], left_on='ann_city', right_on='Town', how='left').drop(columns=['Town'])

# Заменяем значения в type_city по условиям Count
df_test.loc[(df_test['Count'] >= 50) & (df_test['Count'] < 100), 'type_city'] = 'gor_100'
df_test.loc[(df_test['Count'] >= 100) & (df_test['Count'] < 250), 'type_city'] = 'gor_250'
df_test.loc[(df_test['Count'] >= 250) & (df_test['Count'] < 500), 'type_city'] = 'gor_500'
df_test.loc[(df_test['Count'] >= 500) & (df_test['Count'] < 1000), 'type_city'] = 'gor_1000'
df_test.loc[(df_test['Count'] >= 1000) & (df_test['Count'] < 2500), 'type_city'] = 'gor_2500'
df_test.loc[(df_test['Count'] >= 2500) & (df_test['Count'] < 5000), 'type_city'] = 'gor_5000'
df_test.loc[df_test['Count'] > 5000, 'type_city'] = 'gor_10000'
# df_test = df_test.drop(columns=['Count'])

In [26]:
import joblib
# Загрузка (если нужно)
rf_pipeline = load("rf_pipeline.joblib")

# Обучение Pipeline
transformed_train = rf_pipeline.fit_transform(df_train)

# Преобразование тестовых данных
transformed_test = rf_pipeline.transform(df_test)

In [27]:
# Преобразуем и разделяем признаки и таргет
X_train = transformed_train.copy()
y_train = transformed_train['car_price']
X_test = transformed_test.copy()
y_test = transformed_test['car_price']
X_train = X_train.drop(['car_price'], axis=1)
X_test = X_test.drop(['car_price'], axis=1)

In [28]:
rf_model = joblib.load('random_forest_pipeline_without_log.pkl')

In [29]:
def replace_negative_nonlog(y_pred, threshold=50000):
    return np.maximum(y_pred, threshold)

In [30]:
y_pred_rf = replace_negative_nonlog(rf_model.predict(X_test))
y_pred_train_rf = replace_negative_nonlog(rf_model.predict(X_train))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 12, 13, 14, 15, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 10, 12, 13, 14, 15, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [31]:
# Функция вывода основных характеристик модели (ориентируемся прежде всего на R^2)
def print_r2_mse(y_train, y_pred_train, y_test, y_pred):
    '''
    Функция принимает на вход признаки и таргет для тренировочных
    и тестовых данных, модель и выводит на экран 
    значения MSE, R^2 для трейна и теста для указанных параметров
    '''
    print('*'*20)
    print(f'Значение MAE для трейна:  {MAE(y_train, y_pred_train)}')
    print(f'Значение MAE для теста:   {MAE(y_test, y_pred)}')
    print('*'*20)
    print(f'Значение MSE для трейна:  {MSE(y_train, y_pred_train)}')
    print(f'Значение MSE для теста:   {MSE(y_test, y_pred)}')
    print('*'*20)
    print(f'Значение R^2 для трейна:  {r2_score(y_train, y_pred_train)}')
    print(f'Значение R^2 для теста:   {r2_score(y_test, y_pred)}')

In [32]:
print_r2_mse(y_train, y_pred_train_rf, y_test, y_pred_rf)

********************
Значение MAE для трейна:  318444.5331730101
Значение MAE для теста:   342453.2955828191
********************
Значение MSE для трейна:  3754516996520.8765
Значение MSE для теста:   1590771117878.0056
********************
Значение R^2 для трейна:  0.9069304226634172
Значение R^2 для теста:   0.9570446456932187


In [33]:
# Загрузка (если нужно)
loaded_pipeline = load("pipeline.joblib")

# Обучение Pipeline
transformed_train = loaded_pipeline.fit_transform(df_train)

# Преобразование тестовых данных
transformed_test = loaded_pipeline.transform(df_test)

In [34]:
# Преобразуем и разделяем признаки и таргет
X_train = transformed_train.copy()
y_train = transformed_train['car_price']
X_test = transformed_test.copy()
y_test = transformed_test['car_price']
X_train = X_train.drop(['car_price'], axis=1)
X_test = X_test.drop(['car_price'], axis=1)

In [35]:
# Преобразование категориальных признаков в Pandas category
for col in X_train.select_dtypes(include=['object']).columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [36]:


# Загрузка модели
lgbm_model = joblib.load('lgbm_pipeline_.pkl')
catboost_model = joblib.load('catboost_pipeline_.pkl')
ridge_model = joblib.load('ridge_pipeline_.pkl')


In [38]:
def replace_negative(y_pred, threshold=50000):
    delta_y = 2900000
    return np.log(np.maximum(np.exp(y_pred) - delta_y, threshold) + delta_y)

In [39]:
threshold = 50000
y_pred_lgbm = replace_negative(lgbm_model.predict(X_test))
y_pred_train_lgbm = replace_negative(lgbm_model.predict(X_train))

y_pred_cb = replace_negative(catboost_model.predict(X_test))
y_pred_train_cb = replace_negative(catboost_model.predict(X_train))

y_pred_ridge = replace_negative(ridge_model.predict(X_test))
y_pred_train_ridge = replace_negative(ridge_model.predict(X_train))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [40]:
y_pred = (y_pred_lgbm + y_pred_cb + np.log(y_pred_rf + 2900000) + y_pred_ridge) / 4
y_pred_train = (y_pred_train_lgbm + y_pred_train_cb + np.log(y_pred_train_rf + 2900000) + y_pred_train_ridge) / 4

In [41]:
# Функция вывода основных характеристик модели (ориентируемся прежде всего на R^2)
def print_r2_mse_exp(y_train, y_pred_train, y_test, y_pred):
    '''
    Функция принимает на вход признаки и таргет для тренировочных
    и тестовых данных, модель и выводит на экран 
    значения MSE, R^2 для трейна и теста для указанных параметров
    '''

    delta_y = 2900000
    
    print('*'*20)
    print(f'Значение MAE для трейна:  {MAE(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение MAE для теста:   {MAE(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')
    print('*'*20)
    print(f'Значение MSE для трейна:  {MSE(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение MSE для теста:   {MSE(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')
    print('*'*20)
    print(f'Значение R^2 для трейна:  {r2_score(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение R^2 для теста:   {r2_score(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')

In [42]:
print_r2_mse_exp(y_train, y_pred_train_lgbm, y_test, y_pred_lgbm) # Выводим результаты, довольны собой

********************
Значение MAE для трейна:  217883.5061278518
Значение MAE для теста:   315296.57878201717
********************
Значение MSE для трейна:  522908559701.8059
Значение MSE для теста:   1875553347889.8586
********************
Значение R^2 для трейна:  0.9870377791118736
Значение R^2 для теста:   0.9493547137772103


In [43]:
print_r2_mse_exp(y_train, y_pred_train_cb, y_test, y_pred_cb)

********************
Значение MAE для трейна:  284409.0440363049
Значение MAE для теста:   329627.4264487732
********************
Значение MSE для трейна:  986581755521.0713
Значение MSE для теста:   1488183431983.5361
********************
Значение R^2 для трейна:  0.9755439256023036
Значение R^2 для теста:   0.9598148056147716


In [44]:
print_r2_mse_exp(y_train, y_pred_train_ridge, y_test, y_pred_ridge)

********************
Значение MAE для трейна:  320237.2176858907
Значение MAE для теста:   349570.44352475257
********************
Значение MSE для трейна:  1522427337678.8303
Значение MSE для теста:   1606004172879.373
********************
Значение R^2 для трейна:  0.9622610128081118
Значение R^2 для теста:   0.9566333097898919


In [45]:
print_r2_mse_exp(y_train, y_pred_train, y_test, y_pred) # Выводим результаты, довольны собой

********************
Значение MAE для трейна:  260343.44703678312
Значение MAE для теста:   304948.6057331881
********************
Значение MSE для трейна:  1289573884778.0269
Значение MSE для теста:   1384649183522.8447
********************
Значение R^2 для трейна:  0.9680331460713049
Значение R^2 для теста:   0.9626105254235696
